# Pyspark Compatibility Tests

## PCA
From: https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.feature.PCA.html#pyspark.ml.feature.PCA

In [ ]:
PYSPARK = False
SPARK_RAPIDS_ML = not PYSPARK

import shutil

In [ ]:
if PYSPARK:
    from pyspark.ml.feature import PCA, PCAModel
else:
    from spark_rapids_ml.feature import PCA, PCAModel

from pyspark.ml.linalg import Vectors

In [ ]:
data = [(Vectors.sparse(5, [(1, 1.0), (3, 7.0)]),),
        (Vectors.dense([2.0, 0.0, 3.0, 4.0, 5.0]),),
        (Vectors.dense([4.0, 0.0, 0.0, 6.0, 7.0]),)]

data

In [ ]:
df = spark.createDataFrame(data,["features"])
df.show(); df.schema

In [ ]:
pca = PCA()
print(pca.explainParams())

In [ ]:
pca = PCA(k=2, inputCol="features")
pca.setOutputCol("pca_features")

In [ ]:
print(pca.explainParams())

In [ ]:
if SPARK_RAPIDS_ML:
    print(pca._param_mapping())
    print(pca.cuml_params)

In [ ]:
pca.setK(3)

In [ ]:
print(pca.explainParams())

In [ ]:
if SPARK_RAPIDS_ML:
    print(pca.cuml_params)

In [ ]:
pca.setK(2)

In [ ]:
model = pca.fit(df)

In [ ]:
model.getK()
# 2

In [ ]:
model.setOutputCol("output")

In [ ]:
print(model.explainParams())

In [ ]:
if SPARK_RAPIDS_ML:
    print(model.cuml_params)

In [ ]:
model.transform(df).collect()[0].output
# DenseVector([1.648..., -4.013...])

In [ ]:
model.explainedVariance
# DenseVector([0.794..., 0.205...])

In [ ]:
model.pc
# DenseMatrix(5, 2, [-0.4486, 0.133, -0.1252, 0.2165, -0.8477, -0.2842, -0.0562, 0.7636, -0.5653, -0.1156], 0)

In [ ]:
pcaPath = "/tmp/pca"
pca.write().overwrite().save(pcaPath)

In [ ]:
loadedPca = PCA.load(pcaPath)
loadedPca.getK() == pca.getK()
# True

In [ ]:
# confirm saved estimator cuml_params
if SPARK_RAPIDS_ML:
    print(pca.cuml_params)
    print(loadedPca.cuml_params)

In [ ]:
modelPath = "/tmp/pca-model"
model.write().overwrite().save(modelPath)

In [ ]:
loadedModel = PCAModel.load(modelPath)
loadedModel.pc == model.pc
# True

In [ ]:
# confirm saved model cuml_params
if SPARK_RAPIDS_ML:
    print(model.cuml_params)
    print(loadedModel.cuml_params)

In [ ]:
loadedModel.explainedVariance == model.explainedVariance
# True

In [ ]:
loadedModel.transform(df).take(1) == model.transform(df).take(1)
# True

## KMeans
From: https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.clustering.KMeans.html#pyspark.ml.clustering.KMeans

In [ ]:
PYSPARK = False
SPARK_RAPIDS_ML = not PYSPARK

import shutil

In [ ]:
if PYSPARK:
    from pyspark.ml.clustering import KMeans, KMeansModel
else:
    from spark_rapids_ml.clustering import KMeans, KMeansModel

In [ ]:
from pyspark.ml.linalg import Vectors

In [ ]:
data = [(Vectors.dense([0.0, 0.0]), 2.0), (Vectors.dense([1.0, 1.0]), 2.0),
        (Vectors.dense([9.0, 8.0]), 2.0), (Vectors.dense([8.0, 9.0]), 2.0)]

In [ ]:
df = spark.createDataFrame(data, ["features", "weighCol"]).repartition(1)
df.show(); df.schema

In [ ]:
kmeans = KMeans()

In [ ]:
print(kmeans.explainParams())

In [ ]:
if SPARK_RAPIDS_ML:
    print(kmeans._param_mapping())
    print(kmeans.cuml_params)

In [ ]:
kmeans = KMeans(k=2)
kmeans.setSeed(1)
kmeans.setMaxIter(10)

if PYSPARK:
    kmeans.setWeightCol("weighCol")

In [ ]:
print(kmeans.explainParams())

In [ ]:
if SPARK_RAPIDS_ML:
    print(kmeans.cuml_params)

In [ ]:
kmeans.getMaxIter()
# 10

In [ ]:
kmeans.clear(kmeans.maxIter)

In [ ]:
print(kmeans.explainParams())

In [ ]:
if SPARK_RAPIDS_ML:
    print(kmeans.cuml_params)

In [ ]:
kmeans.setFeaturesCol("features")

In [ ]:
model = kmeans.fit(df)

In [ ]:
model.getDistanceMeasure()
# 'euclidean'
# Note: this is not used in spark_rapids_ml (may be implied)

In [ ]:
model.setPredictionCol("newPrediction")

In [ ]:
print(model.explainParams())

In [ ]:
if SPARK_RAPIDS_ML:
    print(model.cuml_params)

In [ ]:
if PYSPARK:
    model.predict(df.head().features)
    # 0
else:
    # NotImplementedError: 'predict' method is not supported, use 'transform' instead.
    pass

In [ ]:
centers = model.clusterCenters()
len(centers)
# 2

In [ ]:
centers
# [array([0.5, 0.5]), array([8.5, 8.5])]

In [ ]:
if PYSPARK:
    transformed = model.transform(df).select("features", "newPrediction")
else:
    # AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `features` cannot be resolved. Did you mean one of the following? [`prediction`].;
    # 'Project ['features, 'newPrediction]
    # +- MapInPandas _transform_udf(weighCol#1, features#29)#35, [prediction#36]
    #    +- Project [weighCol#1, features#29]
    #       +- Project [cuml_values_c3BhcmtjdW1sCg==#26, weighCol#1, UDF(cuml_values_c3BhcmtjdW1sCg==#26) AS features#29]
    #          +- Project [features#0 AS cuml_values_c3BhcmtjdW1sCg==#26, weighCol#1]
    #             +- Repartition 1, true
    #                +- LogicalRDD [features#0, weighCol#1], false    
    transformed = model.transform(df)
    
rows = transformed.collect()

In [ ]:
transformed = model.transform(df)
transformed.show()
# +---------+--------+-------------+
# | features|weighCol|newPrediction|
# +---------+--------+-------------+
# |[0.0,0.0]|     2.0|            0|
# |[1.0,1.0]|     2.0|            0|
# |[9.0,8.0]|     2.0|            1|
# |[8.0,9.0]|     2.0|            1|
# +---------+--------+-------------+

In [ ]:
rows[0].newPrediction == rows[1].newPrediction
# True

In [ ]:
rows[2].newPrediction == rows[3].newPrediction
# True

In [ ]:
model.hasSummary
# True

In [ ]:
if PYSPARK:
    summary = model.summary
    summary.k
    # 2

In [ ]:
if PYSPARK:
    summary.clusterSizes
    # [2, 2]

In [ ]:
if PYSPARK:
    summary.trainingCost
    # 4.0

In [ ]:
kmeans_path = "/tmp/kmeans"
kmeans.write().overwrite().save(kmeans_path)

In [ ]:
kmeans2 = KMeans.load(kmeans_path)
kmeans2.getK()
# 2

In [ ]:
# confirm saved estimator cuml_params
if SPARK_RAPIDS_ML:
    print(kmeans.cuml_params)
    print(kmeans2.cuml_params)

In [ ]:
model_path = "/tmp/kmeans_model"
model.write().overwrite().save(model_path)

In [ ]:
model2 = KMeansModel.load(model_path)

In [ ]:
# confirm saved model cuml_params
if SPARK_RAPIDS_ML:
    print(model.cuml_params)
    print(model2.cuml_params)

In [ ]:
model2.hasSummary
# False

In [ ]:
model.clusterCenters()[0] == model2.clusterCenters()[0]
# array([ True,  True], dtype=bool)

In [ ]:
model.clusterCenters()[1] == model2.clusterCenters()[1]
# array([ True,  True], dtype=bool)

In [ ]:
model.transform(df).take(1) == model2.transform(df).take(1)
# True

In [ ]:
model.transform(df).take(1)
# [Row(features=DenseVector([0.0, 0.0]), weighCol=2.0, newPrediction=0)]

In [ ]:
df.take(1)
# [Row(features=DenseVector([0.0, 0.0]), weighCol=2.0)]

## LinearRegression

From: https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.regression.LinearRegression.html#pyspark.ml.regression.LinearRegression

In [ ]:
PYSPARK = False
SPARK_RAPIDS_ML = not PYSPARK

import shutil

In [ ]:
if PYSPARK:
    from pyspark.ml.regression import LinearRegression, LinearRegressionModel
else:
    from spark_rapids_ml.regression import LinearRegression, LinearRegressionModel

from pyspark.ml.linalg import Vectors

In [ ]:
# Note: spark_rapids_ml.regression.LinearRegression doesn't support datasets with only one feature, so padding dataset
df = spark.createDataFrame([
    (1.0, 2.0, Vectors.dense(1.0, 0.0)),
    (0.0, 2.0, Vectors.sparse(2, [], []))], ["label", "weight", "features"])

df.show(); df.schema

In [ ]:
if PYSPARK:
    lr = LinearRegression(regParam=0.0, solver="normal", weightCol="weight")
else:
    # 'solver: normal' gets value mapped to 'solver: eig'
    # 'weightCol` is explicitly not supported
    lr = LinearRegression(regParam=0.0, solver="normal")

lr.setMaxIter(5)
lr.getMaxIter()
# 5

In [ ]:
print(lr.explainParams())

In [ ]:
if SPARK_RAPIDS_ML:
    print(lr.cuml_params)

In [ ]:
lr.setRegParam(0.1)
lr.getRegParam()
# 0.1

In [ ]:
lr.setRegParam(0.0)

In [ ]:
print(lr.explainParams())

In [ ]:
if SPARK_RAPIDS_ML:
    print(lr.cuml_params)

In [ ]:
model = lr.fit(df)

In [ ]:
model.setFeaturesCol("features")
model.setPredictionCol("newPrediction")
model.getMaxIter()
# 5

In [ ]:
model.getMaxBlockSizeInMB()
# 0.0

In [ ]:
# Note: spark_rapids_ml.regression.LinearRegression doesn't supports datasets with only one feature, so padding dataset
test0 = spark.createDataFrame([(Vectors.dense(1.0, 1.0),)], ["features"])

In [ ]:
if PYSPARK:
    print(abs(model.predict(test0.head().features) - (1.0)) < 0.001)
    # True
else:
    # NotImplementedError: 'predict' method is not supported, use 'transform' instead.
    pass

In [ ]:
abs(model.transform(test0).head().newPrediction - (1.0)) < 0.001
# True

In [ ]:
abs(model.coefficients[0] - 1.0) < 0.001
# True

In [ ]:
model.coefficients
# DenseVector([1.0, 0.0])

In [ ]:
abs(model.intercept - 0.0) < 0.001
# True

In [ ]:
lr_path = "/tmp/lr"
lr.write().overwrite().save(lr_path)

In [ ]:
lr2 = LinearRegression.load(lr_path)
lr2.getMaxIter()
# 5

In [ ]:
model_path = "/tmp/lr_model"
model.write().overwrite().save(model_path)

In [ ]:
model2 = LinearRegressionModel.load(model_path)
model.coefficients[0] == model2.coefficients[0]
# True

In [ ]:
model.intercept == model2.intercept
# True

In [ ]:
model.transform(test0).take(1) == model2.transform(test0).take(1)
# True

In [ ]:
model.numFeatures
# 2